In [7]:
from model import MusicTransformer
import custom
from custom.metrics import *
from custom.criterion import SmoothCrossEntropyLoss, CustomSchedule
from custom.config import config
from data import Data
from midi_processor.processor import encode_midi, decode_midi, RANGE_VEL
from extra import *

import os
from preprocess import preprocess_midi_files_under
from progress.bar import Bar
import pickle

import utils
import datetime
import time

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter

get_config(config, ["config/thor_basic.yml"])

# load data
dataset = Data(config.pickle_dir)
print(dataset)

<class Data has "1008" files>


In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.01
# ------------
vocab_size = START_IDX['velocity'] + RANGE_VEL + 1

torch.manual_seed(1337)

data = torch.tensor(dataset.all_data())
# data = torch.tensor(dataset.batch(1,2000)[2], dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# model = BigramLanguageModel()
# m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



0.908165 M parameters
step 0: train loss 2.6907, val loss 2.6964
step 100: train loss 2.6951, val loss 2.6952
step 200: train loss 2.6916, val loss 2.6836
step 300: train loss 2.6888, val loss 2.6985
step 400: train loss 2.6853, val loss 2.6898
step 500: train loss 2.6793, val loss 2.6829
step 600: train loss 2.6850, val loss 2.6889
step 700: train loss 2.6734, val loss 2.6764
step 800: train loss 2.6768, val loss 2.6639
step 900: train loss 2.6697, val loss 2.6772
step 1000: train loss 2.6767, val loss 2.6698
step 1100: train loss 2.6697, val loss 2.6668
step 1200: train loss 2.6609, val loss 2.6707
step 1300: train loss 2.6540, val loss 2.6768
step 1400: train loss 2.6557, val loss 2.6611
step 1500: train loss 2.6653, val loss 2.6527
step 1600: train loss 2.6616, val loss 2.6656
step 1700: train loss 2.6541, val loss 2.6604
step 1800: train loss 2.6505, val loss 2.6532
step 1900: train loss 2.6511, val loss 2.6516
step 2000: train loss 2.6430, val loss 2.6488
step 2100: train loss 2.

In [26]:
torch.save(model.state_dict(), 'models/event_predicter_15000.pth')

In [39]:
# model = BigramLanguageModel()
# model.load_state_dict(torch.load('models/event_predicter_10000.pth', weights_only=True))
# m = model.to(device)

c:\Users\Draco\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [27]:
# generate from the model
context = torch.tensor([[313,
 57,
 313,
 66,
 287,
 273,
 185,
 194,
 313,
 66,
 287,
 273,
 194,
 313,
 67,
 313,
 50,
 287,
 273,
 195,
 178,
 313,
 69,
 287,
 273,
 197,
 313,
 69,
 313,
 47,
 287,
 273,
 197,
 175,
 313,
 67,
 280,
 313,
 49,
 280,
 195,
 177,
 313,
 66,
 313,
 50,
 287,
 273,
 194,
 178,
 313,
 64,
 313,
 55,
 287,
 273,
 192,
 183,
 313,
 62,
 287,
 273,
 190,
 313,
 62,
 313,
 54,
 287,
 273,
 190,
 182,
 313,
 64,
 313,
 52,
 287,
 273,
 192,
 180,
 313,
 66,
 313,
 50,
 287,
 273,
 194,
 178,
 313,
 66,
 313,
 50,
 287,
 273,
 194,
 178,
 313,
 49,
 280,
 313,
 64,
 280,
 177,
 192,
 313,
 47,
 313,
 64,
 287,
 273,
 175,
 313,
 45,
 287,
 273,
 192,
 173,
 313,
 66,
 313,
 57,
 287,
 273,
 194,
 185,
 313,
 66,
 287,
 273,
 194,
 313,
 67,
 313,
 50,
 287,
 273,
 195,
 178,
 313,
 69,
 287,
 273,
 197,
 313,
 69,
 313,
 47,
 287,
 273,
 197,
 175,
 313,
 67,
 280,
 313,
 49,
 280,
 195,
 177,
 313,
 66,
 313,
 50,
 287,
 273,
 194,
 178,
 313,
 64,
 313,
 55,
 287,
 273,
 192,
 183,
 313,
 62,
 287,
 273,
 190,
 313,
 54,
 313,
 62,
 280,
 182,
 313,
 52,
 280,
 190,
 180,
 313,
 50,
 313,
 64,
 280,
 178,
 313,
 49,
 280,
 192,
 177,
 313,
 47,
 313,
 66,
 280,
 175,
 313,
 45,
 280,
 194,
 173,
 313,
 64,
 313,
 43,
 287,
 273,
 192,
 280,
 171,
 313,
 62,
 313,
 45,
 280,
 190,
 173,
 313,
 62,
 313,
 42,
 287,
 287,
 287,
 259,
 190,
 170]], device=device)
output_sequence = model.generate(context, max_new_tokens=1000)[0].tolist()

In [28]:
decode_midi(output_sequence, file_path=f"latest_midi.midi")

info removed pitch: 37
info removed pitch: 82
info removed pitch: 70
info removed pitch: 90
info removed pitch: 59
info removed pitch: 51
info removed pitch: 40
info removed pitch: 74
info removed pitch: 78
info removed pitch: 84
info removed pitch: 76
info removed pitch: 75
info removed pitch: 88


tensor([256, 381,  67, 378,  65, 379,  79, 273, 195, 193, 207, 256, 381,  67,
        378,  65, 379,  79, 273, 195, 193, 207, 256, 381,  67, 378,  65, 379,
         79, 273, 195, 193, 207, 256, 381,  67, 378,  65, 379,  79, 273, 195,
        193, 207, 256, 381,  67, 378,  65, 379,  79, 273, 195, 193, 207, 256,
        381,  67, 378,  64, 379,  79, 273, 195, 192, 207, 256, 381,  67, 378,
         64, 379,  79, 273, 195, 192, 207, 256, 381,  67, 378,  64, 379,  79,
        273, 195, 192, 207, 256, 381,  67, 378,  64, 379,  79, 273, 195, 192,
        207, 256])

In [30]:
data

tensor([16, 11,  0, 14, 22,  0, 13,  5, 21,  8,  0,  3,  0, 13, 16, 20,  9,  0,
         7, 11,  9,  9, 18,  9,  1,  0,  7, 11,  9,  9, 18,  9,  0, 12, 18,  0,
        19, 11,  9,  0,  6,  9, 18, 19,  0, 19, 11, 12, 15, 10,  0, 19, 11,  9,
        17,  9,  0,  9, 20,  9, 17,  0, 21,  5, 18,  2,  2,  0,  4,  4, 11, 11,
        11, 11, 11,  0,  7, 11,  9,  9, 18,  9])

In [12]:
list(utils.find_files_by_extensions('dataset/preprocessed_midi', ['.pickle']))
batch_x, batch_y = dataset.slide_seq2seq_batch(config.batch_size, config.max_seq)